## Getting Started {#getting-started}

First, install PySpark:

```bash
pip install pyspark
```

Create a SparkSession to start working with PySpark:

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("PySpark SQL Guide")
    .getOrCreate()
)

## Setup: Create Sample Data

First, let's create a sample CSV file that we'll use in this tutorial:

In [ ]:
import csv

# Create sample data.csv file
data = [
    ["customer_id", "name", "region", "amount"],
    [1, "Alice", "North", 100],
    [2, "Bob", "South", 150],
    [3, "Charlie", "North", 200],
    [4, "Diana", "East", 120],
    [5, "Eve", "West", 180]
]

with open("data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(data)

The SparkSession is your entry point to all PySpark functionality.

## Creating DataFrames {#creating-dataframes}

PySpark supports creating DataFrames from multiple sources including Python objects, pandas DataFrames, files, and databases.

Create from Python dictionaries:

In [ ]:
data = {
    "customer_id": [1, 2, 3, 4, 5],
    "name": ["Alice", "Bob", "Charlie", "Diana", "Eve"],
    "region": ["North", "South", "North", "East", "West"],
    "amount": [100, 150, 200, 120, 180]
}

# Create DataFrame from Python dictionary using zip and tuples
df = spark.createDataFrame(
    [(k, v1, v2, v3) for k, v1, v2, v3 in zip(
        data["customer_id"],
        data["name"],
        data["region"],
        data["amount"]
    )],
    ["customer_id", "name", "region", "amount"]
)

Convert from pandas:

In [ ]:
import pandas as pd

pandas_df = pd.DataFrame(data)
# Convert pandas DataFrame to PySpark DataFrame
df = spark.createDataFrame(pandas_df)
df.show()

Load from CSV files:

In [ ]:
# Load CSV file with automatic schema inference
df = spark.read.csv("data.csv", header=True, inferSchema=True)

Read with schema specification:

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

schema = StructType([
    StructField("customer_id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("region", StringType(), True),
    StructField("amount", LongType(), True)
])

# Load CSV file with explicit schema definition
df = spark.read.csv("data.csv", header=True, schema=schema)

## Understanding Lazy Evaluation {#understanding-lazy-evaluation}

PySpark's execution model differs fundamentally from pandas. Operations are divided into two types.

Transformations are lazy operations that build execution plans without running:

In [ ]:
# Transformations return immediately
filtered = df.filter(df.amount > 100)
print(f"Filtered: {filtered}")

selected = filtered.select("name", "amount")
print(f"Selected: {selected}")

Common transformations include `select()`, `filter()`, `withColumn()`, and `groupBy()`. They return instantly because they only build an execution plan and can be chained without performance cost.

Actions trigger execution and return actual results:

In [ ]:
selected.show()

Common actions include `show()`, `collect()`, `count()`, and `describe()`. They execute the entire chain of transformations and return actual results.

This lazy evaluation enables Spark's [Catalyst optimizer](https://www.databricks.com/glossary/catalyst-optimizer) to analyze your complete workflow and apply optimizations like predicate pushdown and column pruning before execution.

## Data Exploration {#data-exploration}

Data exploration in PySpark works similarly to pandas, but with methods designed for distributed computing. Instead of pandas' `df.info()` and `df.head()`, PySpark uses `printSchema()` and `show()` to inspect schemas and preview records across the cluster.

View the schema:

In [ ]:
df.printSchema()

Preview the first few rows:

In [ ]:
# Display the first 5 rows of the DataFrame
df.show(5)

Get summary statistics:

In [ ]:
# Generate summary statistics for numeric columns
df.describe().show()

Count total rows:

In [ ]:
df.count()

List column names:

In [ ]:
df.columns

Get distinct values in a column:

In [ ]:
# Get all unique values in the region column
df.select("region").distinct().show()

Sample random rows:

In [ ]:
# Randomly sample 60% of the rows
df.sample(fraction=0.6, seed=42).show()

## Selection & Filtering {#selection-filtering}

When selecting and filtering data, PySpark uses explicit methods like `select()` and `filter()` that build distributed execution plans.

Select specific columns:

In [ ]:
# Select columns name and amount
df.select("name", "amount").show()

Filter rows with conditions:

In [ ]:
# Filter rows where amount is greater than 150
df.filter(df.amount > 150).show()

Chain multiple filters:

In [ ]:
# Get rows where amount is greater than 100 and region is North
(
    df.filter(df.amount > 100)
  .filter(df.region == "North")
  .show()
)

Drop columns:

In [ ]:
# Drop the customer_id column
df.drop("customer_id").show()

## Column Operations {#column-operations}

Unlike pandas' mutable operations where `df['new_col']` modifies the DataFrame in place, PySpark's `withColumn()` and `withColumnRenamed()` return new DataFrames, maintaining the distributed computing model.

The `withColumn()` method takes two arguments: the new column name and an expression defining its values:

In [ ]:
from pyspark.sql.functions import col

# Add a new column with the amount with tax
df.withColumn("amount_with_tax", col("amount") * 1.1).show()

Add constant value columns with `lit()`:

In [ ]:
from pyspark.sql.functions import lit

# Add a column with the same value for all rows
df.withColumn("country", lit("USA")).select("name", "amount", "country").show()

Use `withColumnRenamed()` to rename a column by specifying the old name and new name:

In [ ]:
# Rename the amount column to revenue
df.withColumnRenamed("amount", "revenue").show()

Use the `cast()` method to convert a column to a different data type:

In [ ]:
# Cast the amount column to a string
df.withColumn("amount_str", col("amount").cast("string")).printSchema()

## Aggregation Functions {#aggregation-functions}

Unlike pandas' in-memory aggregations, PySpark's `groupBy()` and aggregation functions distribute calculations across cluster nodes, using the same conceptual model as pandas but with lazy evaluation.

Apply aggregation functions directly in `select()` to compute values across all rows without grouping:

In [ ]:
from pyspark.sql.functions import sum, avg, count, max, min

# Calculate total revenue, average revenue, and count across all rows
df.select(
    sum("amount").alias("total_revenue"),
    avg("amount").alias("avg_revenue"),
    count("*").alias("order_count")
).show()

Combine `groupBy()` to create groups and `agg()` to compute multiple aggregations per group:

In [ ]:
# Calculate total revenue and customer count by region
(
    df.groupBy("region")
    .agg(
        sum("amount").alias("total_revenue"),
        count("*").alias("customer_count")
    )
    .show()
)

Combine `groupBy()` with `collect_list()` to create arrays of values for each group:

In [ ]:
from pyspark.sql.functions import collect_list

# Collect customer names into an array for each region
(
    df.groupBy("region")
    .agg(collect_list("name").alias("customers"))
    .show(truncate=False)
)

## String Functions {#string-functions}

Unlike pandas' vectorized string methods accessed via `.str`, PySpark provides importable functions like `concat()`, `split()`, and `regexp_replace()` that transform entire columns across distributed partitions.

Use `concat()` to combine multiple columns and literal strings, wrapping constant values with `lit()`:

In [ ]:
from pyspark.sql.functions import concat, lit

# Concatenate customer name and region with a separator
df.withColumn("full_info", concat(col("name"), lit(" - "), col("region"))).show()

Use `split()` to divide a string column into an array based on a delimiter pattern:

In [ ]:
from pyspark.sql.functions import split

# Create sample data with email addresses
email_data = spark.createDataFrame(
    [("alice@company.com",), ("bob@startup.io",), ("charlie@corp.net",)],
    ["email"]
)

# Split email into username and domain
(
    email_data.withColumn("email_parts", split(col("email"), "@"))
    .select("email", "email_parts")
    .show(truncate=False)
)

Use `regexp_replace()` to find and replace text patterns using regular expressions:

In [ ]:
from pyspark.sql.functions import regexp_replace

# Create sample data with phone numbers
phone_data = spark.createDataFrame(
    [("Alice", "123-456-7890"), ("Bob", "987-654-3210"), ("Charlie", "555-123-4567")],
    ["name", "phone"]
)

# Mask phone numbers, keeping only last 4 digits
(
    phone_data.withColumn("masked_phone", regexp_replace(col("phone"), r"\d{3}-\d{3}-(\d{4})", "XXX-XXX-$1"))
    .select("name", "phone", "masked_phone")
    .show()
)

## Date/Time Functions {#datetime-functions}

Working with dates and timestamps is essential for time-based analysis. PySpark offers comprehensive functions to extract date components, format timestamps, and perform temporal operations.

Create sample data with dates:

In [ ]:
from datetime import datetime, timedelta

date_data = [
    (1, datetime(2024, 1, 15), 100),
    (2, datetime(2024, 2, 20), 150),
    (3, datetime(2024, 3, 10), 200),
    (4, datetime(2024, 4, 5), 120),
    (5, datetime(2024, 5, 25), 180)
]

# Create sample DataFrame with datetime values
df_dates = spark.createDataFrame(date_data, ["id", "order_date", "amount"])

Use functions like `year()`, `month()`, and `dayofmonth()` to extract individual date components from timestamp columns:

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth

# Extract year, month, and day components from order_date
(
    df_dates.withColumn("year", year("order_date"))
    .withColumn("month", month("order_date"))
    .withColumn("day", dayofmonth("order_date"))
    .show()
)

Use `date_format()` to convert dates to custom string formats:

In [ ]:
from pyspark.sql.functions import date_format

# Format timestamps as YYYY-MM-DD strings
(
    df_dates.withColumn("formatted_date", date_format("order_date", "yyyy-MM-dd"))
    .select("order_date", "formatted_date")
    .show()
)

Use `to_timestamp()` to convert string columns to timestamp objects by specifying the date format pattern:

In [ ]:
from pyspark.sql.functions import to_timestamp

string_dates = spark.createDataFrame(
    [("2024-01-15",), ("2024-02-20",)],
    ["date_string"]
)

# Convert date strings to timestamp objects
string_dates.withColumn(
    "timestamp",
    to_timestamp("date_string", "yyyy-MM-dd")
).show()

## Working with Time Series {#working-with-time-series}

Time series analysis often requires comparing values across different time periods. PySpark's window functions with lag and lead operations enable calculations of changes and trends over time.

Create sample time series data:

In [ ]:
ts_data = [
    (1, datetime(2024, 1, 1), 100),
    (1, datetime(2024, 1, 2), 120),
    (1, datetime(2024, 1, 3), 110),
    (2, datetime(2024, 1, 1), 200),
    (2, datetime(2024, 1, 2), 220),
    (2, datetime(2024, 1, 3), 210)
]

# Create time series data with multiple dates per customer
df_ts = spark.createDataFrame(ts_data, ["customer_id", "date", "amount"])

Calculate the previous row's value within each customer group using `lag()`:

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

# Create a window: group by customer_id, order by date
window_spec = Window.partitionBy("customer_id").orderBy("date")

# Get the previous amount for each customer
df_ts.withColumn("prev_amount", lag("amount").over(window_spec)).show()

The first row in each customer group has `null` for `prev_amount` because there's no previous value.

Calculate day-over-day change by combining `lag()` to get the previous value and subtracting it from the current value:

In [ ]:
# Calculate the day-over-day change in amount
(
    df_ts.withColumn("prev_amount", lag("amount", 1).over(window_spec))
    .withColumn("daily_change", col("amount") - col("prev_amount"))
    .show()
)

## Window Analytics {#window-analytics}

Complex analytics operations like rankings, running totals, and moving averages require window functions that operate within data partitions. These functions enable sophisticated analytical queries without self-joins.

Apply ranking functions within partitioned groups:

- Combine `Window.partitionBy()` and `Window.orderBy()` to rank within groups
- `rank()` handles ties by giving them the same rank with gaps (e.g., 1, 2, 2, 4)
- `row_number()` always assigns unique sequential numbers (e.g., 1, 2, 3, 4)
- `dense_rank()` gives ties the same rank without gaps (e.g., 1, 2, 2, 3)

In [ ]:
from pyspark.sql.functions import rank, row_number, dense_rank

# Create sample data with categories to show ranking within groups
ranking_data = spark.createDataFrame(
    [("Math", "Alice", 100), ("Math", "Bob", 150), ("Math", "Charlie", 150),
     ("Science", "Diana", 200), ("Science", "Eve", 100)],
    ["subject", "name", "score"]
)

# Define window partitioned by subject, ordered by score descending
window_spec = Window.partitionBy("subject").orderBy(col("score").desc())

# Calculate different ranking methods within each subject
(
    ranking_data.withColumn("rank", rank().over(window_spec))
    .withColumn("row_number", row_number().over(window_spec))
    .withColumn("dense_rank", dense_rank().over(window_spec))
    .show()
)

Calculate running totals using `rowsBetween()` to define a window range:

- `Window.unboundedPreceding` starts the window at the first row of the partition
- `Window.currentRow` ends the window at the current row being processed
- This creates an expanding window that includes all rows from the start up to the current position

In [ ]:
from pyspark.sql.functions import sum as _sum

# Create daily sales data with store identifier
daily_sales = spark.createDataFrame(
    [("A", 1, 50), ("A", 2, 75), ("A", 3, 100),
     ("B", 1, 25), ("B", 2, 150), ("B", 3, 80)],
    ["store", "day", "sales"]
)

# Define window partitioned by store, from beginning to current row
window_spec = (
    Window.partitionBy("store")
    .orderBy("day")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

# Calculate running total of sales per store
daily_sales.withColumn("running_total", _sum("sales").over(window_spec)).show()

Use `rowsBetween(-1, 1)` to create a 3-row sliding window that includes the previous row, current row, and next row:

In [ ]:
# Define window for 3-row moving average (previous, current, next)
window_spec = (
    Window.partitionBy("customer_id")
    .orderBy("date")
    .rowsBetween(-1, 1)
)

# Calculate moving average of amount over the window
df_ts.withColumn("moving_avg", avg("amount").over(window_spec)).show()

## Join Operations {#join-operations}

Combining data from multiple tables is a core operation in data analysis. PySpark supports various join types including inner, left, and broadcast joins, with automatic optimization for performance.

Create sample tables for joining:

In [ ]:
# Create sample customers and orders tables for joining
customers = spark.createDataFrame(
    [(1, "Alice", "US"), (2, "Bob", "UK"), (3, "Charlie", "US")],
    ["customer_id", "name", "country"]
)

orders = spark.createDataFrame(
    [(101, 1, 100), (102, 2, 150), (103, 1, 200), (104, 3, 120)],
    ["order_id", "customer_id", "amount"]
)

Use `join()` to perform an inner join, which returns only rows with matching keys in both DataFrames:

In [ ]:
# Join customers and orders on customer_id
customers.join(orders, "customer_id").show()

Perform a left join by specifying `"left"` as the third argument, which retains all left table rows regardless of matches:

In [ ]:
# Create extended customers table including Diana
customers_extended = spark.createDataFrame(
    [(1, "Alice", "US"), (2, "Bob", "UK"), (3, "Charlie", "US"), (4, "Diana", "CA")],
    ["customer_id", "name", "country"]
)

# Left join to keep all customers even without orders
customers_extended.join(orders, "customer_id", "left").show()

Chain multiple `join()` calls together to combine three or more DataFrames in sequence:

In [ ]:
# Create products and order_items tables
products = spark.createDataFrame(
    [(1, "Widget"), (2, "Gadget")],
    ["product_id", "product_name"]
)

order_items = spark.createDataFrame(
    [(101, 1), (102, 2), (103, 1), (104, 2)],
    ["order_id", "product_id"]
)

# Chain multiple joins to combine orders, items, and products
(
    orders.join(order_items, "order_id")
    .join(products, "product_id")
    .select("order_id", "customer_id", "product_name", "amount")
    .show()
)

## SQL Integration {#sql-integration}

PySpark supports standard SQL syntax for querying data. You can write SQL queries using familiar SELECT, JOIN, and WHERE clauses alongside PySpark operations.

Use `createOrReplaceTempView()` to register a DataFrame as a temporary SQL table, allowing it to be queried multiple times with SQL syntax:

In [ ]:
# Register DataFrame as a temporary SQL view named customers
df.createOrReplaceTempView("customers")

Execute SQL queries on DataFrames registered with `createOrReplaceTempView()` using `spark.sql()`:

In [ ]:
# Execute SQL query to aggregate revenue by region
spark.sql("""
    SELECT region, SUM(amount) as total_revenue
    FROM customers
    GROUP BY region
    ORDER BY total_revenue DESC
""").show()

Chain SQL queries with DataFrame operations by storing `spark.sql()` results and applying methods like `filter()` and `orderBy()`:

In [ ]:
# Query customers with amount greater than 100
result = spark.sql("""
    SELECT customer_id, name, amount
    FROM customers
    WHERE amount > 100
""")

# Filter SQL results using DataFrame API and sort by amount
(
    result.filter(col("region") != "South")
    .orderBy("amount", ascending=False)
    .show()
)

Use SQL syntax within `spark.sql()` for complex joins and aggregations when SQL is more readable than DataFrame API:

In [ ]:
# Register orders and customers as temporary SQL views
orders.createOrReplaceTempView("orders")
customers.createOrReplaceTempView("customers_table")

# Join and aggregate using SQL syntax
spark.sql("""
    SELECT
        c.name,
        COUNT(o.order_id) as order_count,
        SUM(o.amount) as total_spent
    FROM customers_table c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name
    ORDER BY total_spent DESC
""").show()

## Custom Functions {#custom-functions}

When built-in functions aren't sufficient, custom logic can be implemented using pandas UDFs. These user-defined functions provide vectorized performance through Apache Arrow and support both scalar operations and grouped transformations.

Create a scalar pandas UDF with the `@pandas_udf` decorator to apply custom Python functions to columns with vectorized performance:

In [ ]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

# Define pandas UDF to calculate discount based on price and quantity
@pandas_udf("double")
def calculate_discount(amount: pd.Series, quantity: pd.Series) -> pd.Series:
    return amount * quantity * 0.1

# Create sample order data with price and quantity
order_data = spark.createDataFrame(
    [(1, 100.0, 2), (2, 150.0, 3), (3, 200.0, 1)],
    ["order_id", "price", "quantity"]
)

# Apply discount calculation UDF to create discount column
order_data.withColumn(
    "discount",
    calculate_discount(col("price"), col("quantity"))
).show()

Apply custom pandas functions to grouped data with `groupBy().applyInPandas()`:

- Define a function that transforms each group as a pandas DataFrame
- Specify output schema to tell PySpark the resulting column names and types

In [ ]:
# Define function to normalize amounts within each group
def normalize_by_group(pdf: pd.DataFrame) -> pd.DataFrame:
    pdf["normalized"] = (pdf["amount"] - pdf["amount"].mean()) / pdf["amount"].std()
    return pdf

schema = "customer_id long, date timestamp, amount long, normalized double"

# Apply normalization function to each customer_id group
df_ts.groupBy("customer_id").applyInPandas(normalize_by_group, schema).show()

## SQL Expressions {#sql-expressions}

SQL expressions can be embedded directly within DataFrame operations for complex transformations. The `expr()`  and `selectExpr()` functions allow SQL syntax to be used alongside DataFrame methods, providing flexibility in query construction.

Use the `expr()` function to embed SQL syntax within DataFrame operations, allowing SQL-style calculations in `withColumn()`:

In [ ]:
from pyspark.sql.functions import expr

# Add tax and total columns using SQL expressions
(
    df.withColumn("tax", expr("amount * 0.1"))
    .withColumn("total", expr("amount + (amount * 0.1)"))
    .show()
)

Unlike `select()` which uses column objects and method chaining, `selectExpr()` accepts SQL strings and is preferred for complex expressions that are simpler to write as SQL:

In [ ]:
# Select columns with calculations and CASE statement using SQL syntax
df.selectExpr(
    "customer_id",
    "name",
    "amount * 1.1 AS amount_with_tax",
    "CASE WHEN amount > 150 THEN 'high' ELSE 'normal' END as category"
).show()